<a href="https://colab.research.google.com/github/dudgus7477/datascience/blob/main/(%EC%8B%A4%EC%8A%B5%EC%9A%A9)%EC%98%81%ED%99%94%20%EB%A6%AC%EB%B7%B0%20%EA%B0%90%EC%83%81%20%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 영화리뷰에 대한 텍스트와 해당 리뷰가 긍정인 경우 1, 부정인 경우 0으로 표시(label)
- 감성 분류를 수행하는 모델을 만들어보자!

#1. 필요한 라이브러리, 데이터 가져오기

In [ ]:
!pip install konlpy
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 35.1 MB/s eta 0:00:00


In [ ]:
df = pd.read_table('./ratings_test.txt')
df.head()

데이터에 존재하는 영화리뷰 개수를 확인해봅시다.

In [ ]:
print('훈련용 리뷰 개수 :',len(df))

#2. 데이터 전처리
- 중복 유무 확인 및 제거

In [ ]:
# document 열의 중복을 제외한 값의 개수

# document 열의 중복 제거

In [ ]:
#빈 곳의 개수

#빈 부분 제거


In [ ]:
#총 데이터의 개수
print('총 샘플의 수 :', len(df))

- 해당 리뷰의 긍,부정 유무가 기재되어있는 레이블(lable)값의 분포를 확인해보자!

In [ ]:
df['label'].value_counts().plot(kind = 'bar')

- 한글과 공백을 제외하고 모두 제거해보자!(ex. 특수문자)

In [ ]:
# 한글과 공백을 제외하고 모두 제거

In [ ]:
#빈 곳의 개수

#빈 부분 제거


#3. 형태소로 토큰화하기
- 저번 시간에 배운 내용 복습해보자!

In [ ]:
#리뷰 형태소로 토큰화하기


- 기계가 텍스트를 숫자로 처리할 수 있도록 정수 인코딩하기!

- 단어가 26,000개가 넘게 존재!
- 여기서 각 정수는 빈도수가 높은 순서대로 부여됨(즉 높은 정수가 빈도수가 낮음)
> 빈도수가 2이하인 희귀단어를 제외시켜보자!

In [ ]:
#빈도수 체크
hold = 3
total = len(tokenizer.word_index) # 단어의 수
rare = 0   # 등장 빈도수가 hold보다 작은 단어의 개수를 카운트
total_freq = 0   # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0   # 등장 빈도수가 hold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.


    # 단어의 등장 빈도수가 hold보다 작으면


print('단어 집합(vocabulary)의 크기 :',total)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(hold - 1, rare))
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)


In [ ]:
# 등장 빈도가 2회 이하인 단어들은 제외

- 시퀀스 자료형이란?
 리스트와 같이 각각의 요소들이 연속적으로 이어진 자료형



In [ ]:
# 정수 시퀀스로 변환(리스트 형식으로 저장하기위해)

#4. 패딩
- 서로 다른 길이의 샘플들의 길이를 동일하게 맞춰주는 작업

# 5. 모델 만들기

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM #다대일 구조의 LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='          '))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) #손실함수는 크로스 엔트로피 함수
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)


In [ ]:
#정확도 확인


#6. 리뷰 예측하기

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = 30) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측

#긍정, 부정 리뷰 구분
